In [1]:
from datetime import datetime
from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as funcs
import pandas as pd
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window

In [2]:
spark = (SparkSession.builder.appName("pyspark-test-{}".format(datetime.today()))
        .master("spark://192.168.1.11:8081")      
        .getOrCreate())
sqlContext = SQLContext(spark)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
#spark.sparkContext.getConf().getAll()

In [4]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,default,bank_name,False
1,default,pp_transaction,False
2,default,use_case,False
3,default,use_case2,False


In [5]:
pp = spark.sql("SELECT * FROM pp_transaction")
print(pp.count())

8763175


In [8]:
pp.limit(10).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,tps,data_time,bank_send,bank_receive,proxy_type,datatype,date
0,2,2023-05-30 13:07:13,TMB,KBNK,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
1,1,2023-05-30 12:02:26,KBNK,BAY,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
2,6,2023-05-30 13:07:14,BAY,KBNK,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
3,3,2023-05-30 12:02:26,KBNK,BAY,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
4,3,2023-05-30 13:07:14,BAY,KBNK,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
5,1,2023-05-30 12:02:26,KBNK,BAY,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
6,2,2023-05-30 13:07:14,CITI,KBNK,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
7,1,2023-05-30 12:02:26,KBNK,BAY,NATID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
8,1,2023-05-30 13:07:14,CITI,KBNK,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30
9,1,2023-05-30 12:02:26,SCB,BAY,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY,2023-05-30


In [5]:
df_orders = spark.read.format("parquet").load("s3a://pp-transaction/pp_transaction_20230530_parquet_data.parquet", header=True)
df_orders.limit(10).toPandas()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,tps,data_time,bank_send,bank_receive,proxy_type,datatype
0,4,2023-05-30 07:20:02,KTB,SCB,NATID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
1,3,2023-05-30 07:20:02,TMB,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
2,3,2023-05-30 07:20:02,UOBT,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
3,1,2023-05-30 07:20:03,BAAC,SCB,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
4,9,2023-05-30 07:20:03,BBL,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
5,1,2023-05-30 07:20:03,BBL,SCB,MSISDN,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
6,1,2023-05-30 07:20:03,GSB,SCB,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
7,35,2023-05-30 07:20:03,KBNK,SCB,ACCOUNT,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
8,6,2023-05-30 07:20:03,KBNK,SCB,BILLERID,TPS_SWT_LEG1_LEG2_byBANK_andPROXY
9,16,2023-05-30 07:20:03,KBNK,SCB,EWALLET,TPS_SWT_LEG1_LEG2_byBANK_andPROXY


In [5]:
df_orders = spark.read.format("delta").load("s3a://warehouse/bank_name.delta", header=True)
df_orders.limit(10).toPandas()

,bank_send,id
0,MHCB,1
1,TCRB,2
2,GSB,3
3,GHB,4
4,SCB,5
5,SCBT,6
6,CITI,7
7,UOBT,8
8,ISBT,9
9,ICBC,10
